In [ ]:
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 5.5 MB/s eta 0:00:00


In [ ]:
! pip install pytorch_tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 1.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from math import sqrt
import os
import missingno as msno
import calendar
import datetime
from math import sqrt
from collections import defaultdict

#시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

#preprocessing
from sklearn.metrics import mean_squared_error, mean_squared_log_error, make_scorer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

#MModels
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
from sklearn.preprocessing import LabelEncoder


#최적화
# import optuna
# from optuna import Trial
# from optuna.samplers import TPESampler

# #Xai
# from functools import partial 
# import shap

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
train.drop("Unnamed: 0", axis=1, inplace=True)

target_col = "count"
drop_cols = ["datetime", "workingday", "holiday", "Day", "Year", "Hour",target_col] #"sin_hour", "cos_hour"
train_x = train.drop(drop_cols, axis=1)


In [ ]:
test_path = "/content/drive/MyDrive/함께하조/기계학습과 딥러닝/data/kaggle_data/test_eda.csv"
train_path = "/content/drive/MyDrive/함께하조/기계학습과 딥러닝/data/kaggle_data/train_eda.csv"

test = pd.read_csv(test_path)
train = pd.read_csv(train_path)

test.drop("Unnamed: 0", axis=1, inplace=True)
train.drop("Unnamed: 0", axis=1, inplace=True)

#cat features
cat_col = ["season", "Year","weather", "Day of week","Month","Day_info"] #Hour
for col in cat_col:
    train[col] = train[col].astype("category")
    test[col] = test[col].astype("category")

#train, valid split
days = list(range(1, 15))
train_d = train.loc[train['Day'].isin(days)]
test_d = train.loc[~(train['Day'].isin(days))]

#target, drop, y
target_col = "count"
drop_cols = ["datetime", "workingday", "holiday", "Day", "Year", "Hour",target_col] #"sin_hour", "cos_hour"

x_train, y_train = train_d.drop(drop_cols, axis=1), train_d[target_col]
x_test, y_test = test_d.drop(drop_cols, axis=1), test_d[target_col]

x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

x_train.head()

,season,weather,temp,humidity,windspeed,Day of week,Month,Day_info,sin_hour,cos_hour
0,1,Good,9.84,81,0.0,Saturday,1,Weekend,0.000000,1.000000
1,1,Good,9.02,80,0.0,Saturday,1,Weekend,0.258819,0.965926
2,1,Good,9.02,80,0.0,Saturday,1,Weekend,0.500000,0.866025
3,1,Good,9.84,75,0.0,Saturday,1,Weekend,0.707107,0.707107
4,1,Good,9.84,75,0.0,Saturday,1,Weekend,0.866025,0.500000


### 카테고리 컬럼의 Label encoding

In [ ]:
class MultiColLabelEncoder:
    def __init__(self):
        self.encoder_dict = defaultdict(LabelEncoder)
    
    def fit_transform(self, X: pd.DataFrame, columns: list):
        if not isinstance(columns, list):
            columns = [columns]
        
        output = X.copy()
        output[columns] = X[columns].apply(lambda x: self.encoder_dict[x.name].fit_transform(x))

        return output
    
    def inverse_transform(self, X: pd.DataFrame, columns: list):
        if not isinstance(columns, list):
            columns = [columns]
        
        if not all(key in self.encoder_dict for key in columns):
            raise KeyError(f"at leat one of {columns} is not encoded before")
        output = X.copy()
        try:
            output[columns] = X[columns].apply(lambda x:
                                                self.encoder_dict[x.name].inverse_transform(x))
        except ValueError:
            print(f"""Need assignment when do "fit_transform" function""")
            raise
        
        return output

In [ ]:
mcle =  MultiColLabelEncoder()
cat_cols = ["season", "weather", "Day of week", "Month", "Day_info"] #"Hour"
xtrain_t = mcle.fit_transform(x_train, columns=cat_cols)
xtest_t = mcle.fit_transform(x_test, columns=cat_cols)
xtrain_t.head()

,season,weather,temp,humidity,windspeed,Day of week,Month,Day_info,sin_hour,cos_hour
0,0,0,9.84,81,0.0,2,0,0,0.000000,1.000000
1,0,0,9.02,80,0.0,2,0,0,0.258819,0.965926
2,0,0,9.02,80,0.0,2,0,0,0.500000,0.866025
3,0,0,9.84,75,0.0,2,0,0,0.707107,0.707107
4,0,0,9.84,75,0.0,2,0,0,0.866025,0.500000


In [ ]:
## Label encoder 역변환 확인 코드드
# org = mcle.inverse_transform(xtrain_t, columns=cat_cols)
# org.head()

,season,weather,temp,humidity,windspeed,Day of week,Month,Day_info,sin_hour,cos_hour
0,1,Good,9.84,81,0.0,Saturday,1,Weekend,0.000000,1.000000
1,1,Good,9.02,80,0.0,Saturday,1,Weekend,0.258819,0.965926
2,1,Good,9.02,80,0.0,Saturday,1,Weekend,0.500000,0.866025
3,1,Good,9.84,75,0.0,Saturday,1,Weekend,0.707107,0.707107
4,1,Good,9.84,75,0.0,Saturday,1,Weekend,0.866025,0.500000


In [ ]:
categorical_dims = [len(xtrain_t[col].unique()) for col in cat_cols]
categorical_dims

[4, 3, 7, 12, 3]

### 파라미터 세팅

In [ ]:
features = list(xtrain_t.columns)
cat_dims = [len(xtrain_t[col].unique()) for col in cat_cols]
cat_idxs = [ i for i, f in enumerate(features) if f in cat_cols]

print(features)
print(cat_idxs)
print(cat_dims)

In [ ]:
features = list(xtrain_t.columns)
cat_dims = [len(xtrain_t[col].unique()) for col in cat_cols]
cat_idxs = [ i for i, f in enumerate(features) if f in cat_cols]

# define your embedding sizes : here just a random choice(optuna로 설정 뭔지 알아보기)
cat_emb_dim = [5, 4, 3, 6, 2, 2, 1, 10]

print(features)
print(cat_idxs)
print(cat_dims)

['season', 'weather', 'temp', 'humidity', 'windspeed', 'Day of week', 'Month', 'Day_info', 'sin_hour', 'cos_hour']
[0, 1, 5, 6, 7]
[4, 3, 7, 12, 3]


In [ ]:
print(xtrain_t.shape)
print(xtest_t.shape)
print(y_train.shape)
print(y_test.shape)

(8026, 10)
(2860, 10)
(8026,)
(2860,)


In [ ]:
# clf = TabNetRegressor(cat_idxs=cat_idxs,
#                        cat_dims=cat_dims,
#                        cat_emb_dim=5,
#                        optimizer_fn=torch.optim.Adam,
#                        optimizer_params=dict(lr=1e-2),
#                        scheduler_params={"step_size":50,
#                                          "gamma":0.9},
#                        scheduler_fn=torch.optim.lr_scheduler.StepLR,
#                        mask_type='sparsemax' # "sparsemax", entmax
#                       )

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Using {}".format(DEVICE))

Using cpu


#### 아래 코드가 완성 코드

In [ ]:
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score

class RMSLE(Metric):
    def __init__(self):
        self._name = "rmsle_custom"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        y_true = np.expm1(y_true)
        y_pred = np.expm1(y_pred)

        log_true = np.nan_to_num(np.log(y_true+1))
        log_pred = np.nan_to_num(np.log(y_pred+1))

        output = np.sqrt(np.mean((log_true - log_pred)**2))
        return output

clf =  TabNetRegressor(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=5)
clf.fit(
  xtrain_t.values, np.log1p(np.array(y_train).reshape(-1, 1)),
  eval_set=[(xtrain_t.values, np.log1p(np.array(y_train).reshape(-1, 1))),
            (xtest_t.values, np.log1p(np.array(y_test).reshape(-1, 1)))],
            eval_name=['train', 'valid'],
  eval_metric=["rmsle", RMSLE],
  max_epochs=100
)

/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 16.03028| train_rmsle: 0.40713 | train_rmsle_custom: 2.6764  | valid_rmsle: 0.42163 | valid_rmsle_custom: 2.66966 |  0:00:00s
epoch 1  | loss: 4.35383 | train_rmsle: 0.23606 | train_rmsle_custom: 3.36565 | valid_rmsle: 0.23124 | valid_rmsle_custom: 3.40532 |  0:00:01s
epoch 2  | loss: 2.07594 | train_rmsle: 0.14539 | train_rmsle_custom: 2.49883 | valid_rmsle: 0.13749 | valid_rmsle_custom: 2.50963 |  0:00:02s
epoch 3  | loss: 1.47674 | train_rmsle: 0.13011 | train_rmsle_custom: 2.48582 | valid_rmsle: 0.11465 | valid_rmsle_custom: 2.28222 |  0:00:02s
epoch 4  | loss: 1.13025 | train_rmsle: 0.12626 | train_rmsle_custom: 2.18417 | valid_rmsle: 0.11299 | valid_rmsle_custom: 2.06984 |  0:00:03s
epoch 5  | loss: 0.94119 | train_rmsle: 0.1219  | train_rmsle_custom: 1.92414 | valid_rmsle: 0.10198 | valid_rmsle_custom: 1.78172 |  0:00:03s
epoch 6  | loss: 0.79056 | train_rmsle: 0.1144  | train_rmsle_custom: 1.74987 | valid_rmsle: 0.10063 | valid_rmsle_custom: 1.67318 |  0:00:04s

/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
train_rmsle = clf.history["train_rmsle_custom"]
valid_rmsle = clf.history["valid_rmsle_custom"]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(train_rmsle))), y= train_rmsle, name="train"))
fig.add_trace(go.Scatter(x=list(range(len(train_rmsle))), y= valid_rmsle, name="valid"))
fig.show()

In [ ]:
tt = clf.predict(xtest_t.values)
np.expm1(tt)

array([[ 51.57367 ],
       [ 24.112095],
       [ 15.035192],
       ...,
       [140.89789 ],
       [107.95928 ],
       [ 46.677063]], dtype=float32)

In [ ]:
y_test

324       28
325       20
326       12
327        8
328        5
        ... 
10881    336
10882    241
10883    168
10884    129
10885     88
Name: count, Length: 2860, dtype: int64

In [ ]:
np.sqrt(mean_squared_log_error(y_test, np.expm1(tt)))

0.520429200734873

In [ ]:

def Objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    cat_emb_dim = trial.suggest_int("cat_emb_dim", 1, 10)
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type=mask_type, n_shared=n_shared,
                     scheduler_params=dict(mode="min",
                                           patience=trial.suggest_int("patienceScheduler",low=3,high=10),
                                           min_lr=1e-5,
                                           factor=0.5,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=0,
                     cat_idxs = cat_idxs,
                     cat_dims = cat_dims,
                     cat_emb_dim=cat_emb_dim
                     )
    n_splits=5
    random_state=42
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    scores = []

    #local_train, local_valid를 5번 만들어서 수행
    for train_index, valid_index in kf.split(X=xtrain_t, y=y_train):
        X_train, Y_train = xtrain_t.iloc[train_index].values, np.array(y_train[train_index]).reshape(-1, 1)
        X_valid, Y_valid = xtrain_t.iloc[valid_index].values, np.array(y_train[valid_index]).reshape(-1, 1)

        regressor = TabNetRegressor(**tabnet_params)
        regressor.fit(X_train=X_train, y_train=Y_train,
                  eval_set=[(X_valid, Y_valid)],
                  patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
                  eval_metric=["rmsle"])
        
        scores.append(regressor.best_cost)

    avg = np.mean(scores)
    return avg
    
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="TabNet",
    direction="minimize",
    sampler=sampler,
)
study.optimize(Objective, n_trials=20)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

In [ ]:
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score

class RMSLE(Metric):
    def __init__(self):
        self._name = "rmsle_custom"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        y_true = np.expm1(y_true)
        y_pred = np.expm1(y_pred)

        log_true = np.nan_to_num(np.log(y_true+1))
        log_pred = np.nan_to_num(np.log(y_pred+1))

        output = np.sqrt(np.mean((log_true - log_pred)**2))
        return output

def Objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    cat_emb_dim = trial.suggest_int("cat_emb_dim", 1, 10)
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type=mask_type, n_shared=n_shared,
                     scheduler_params=dict(mode="min",
                                           patience=trial.suggest_int("patienceScheduler",low=3,high=10),
                                           min_lr=1e-5,
                                           factor=0.5,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=0,
                     cat_idxs = cat_idxs,
                     cat_dims = cat_dims,
                     cat_emb_dim=cat_emb_dim
                     )
    n_splits=5
    random_state=42
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    scores = []

    #local_train, local_valid를 5번 만들어서 수행
    for train_index, valid_index in kf.split(X=xtrain_t, y=y_train):
        X_train, Y_train = xtrain_t.iloc[train_index].values, np.array(y_train[train_index]).reshape(-1, 1)
        X_valid, Y_valid = xtrain_t.iloc[valid_index].values, np.array(y_train[valid_index]).reshape(-1, 1)

        regressor = TabNetRegressor(**tabnet_params)
        regressor.fit(X_train=X_train, y_train=Y_train,
                  eval_set=[(X_valid, Y_valid)],
                  patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
                  eval_metric=["rmsle"])
        
        scores.append(regressor.best_cost)

    avg = np.mean(scores)
    return avg
    
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="TabNet",
    direction="minimize",
    sampler=sampler,
)
study.optimize(Objective, n_trials=20)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

In [ ]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="TabNet",
    direction="minimize",
    sampler=sampler,
)
study.optimize(Objective, n_trials=20)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2023-03-23 18:04:45,111] A new study created in memory with name: TabNet


Stop training because you reached max_epochs = 3 with best_epoch = 2 and best_val_0_rmsle = 3.49592


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 3 with best_epoch = 2 and best_val_0_rmsle = 2.47297


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


...

Stop training because you reached max_epochs = 70 with best_epoch = 69 and best_val_0_rmsle = 0.15696


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 70 with best_epoch = 67 and best_val_0_rmsle = 0.16215


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-03-23 19:28:32,754] Trial 19 finished with value: 0.16161526546615618 and parameters: {'mask_type': 'entmax', 'n_da': 64, 'n_steps': 2, 'gamma': 1.4, 'n_shared': 2, 'lambda_sparse': 0.0004099227036801607, 'cat_emb_dim': 7, 'patienceScheduler': 4, 'patience': 22, 'epochs': 70}. Best is trial 14 with value: 0.14737630837159596.


Best Score: 0.14737630837159596
Best trial: {'mask_type': 'entmax', 'n_da': 64, 'n_steps': 2, 'gamma': 1.0, 'n_shared': 2, 'lambda_sparse': 7.462098991355312e-05, 'cat_emb_dim': 3, 'patienceScheduler': 4, 'patience': 17, 'epochs': 82}


In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [ ]:
study.best_trial.params

{'mask_type': 'entmax',
 'n_da': 64,
 'n_steps': 2,
 'gamma': 1.0,
 'n_shared': 2,
 'lambda_sparse': 7.462098991355312e-05,
 'cat_emb_dim': 3,
 'patienceScheduler': 4,
 'patience': 17,
 'epochs': 82}

In [ ]:
best_params = {"mask_type": "entmax",
                "n_da": 64,
                "n_steps": 2,
                "gamma": 1.0,
                "n_shared": 2,
                "lambda_sparse": 7.462098991355312e-05,
                "cat_emb_dim": 3,
                "patienceScheduler": 4,
                "patience": 17, "epochs": 82, "cat_idxs": cat_idxs,
                "cat_dims": cat_dims}

In [ ]:
tabnet_params = dict(n_d=64, n_a=64, n_steps=3, gamma=1.0,
                     lambda_sparse=7.462098991355312e-05, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type="entmax", n_shared= 2,
                     scheduler_params=dict(mode="min",
                                           patience=4,
                                           min_lr=1e-5,
                                           factor=0.5,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=0,
                     cat_idxs = cat_idxs,
                     cat_dims = cat_dims,
                     cat_emb_dim=3
                     )

In [ ]:
regressor = TabNetRegressor(**tabnet_params)
regressor.fit(xtrain_t.values, np.array(y_train).reshape(-1, 1))

/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [ ]:
y_pred = regressor.predict(xtest_t.values)
y_pred = np.where(y_pred<0, 0, y_pred)
test_rmsle = mean_squared_log_error(y_test, y_pred, squared=True)
print(test_rmsle)

0.2448928761857799


In [ ]:
y_pred = np.where(y_pred<0, 0, y_pred)

In [ ]:
y_pred[y_pred<0]

array([-1.2504969 , -0.76534986], dtype=float32)

### 제출

In [ ]:
test_path = "/content/drive/MyDrive/함께하조/기계학습과 딥러닝/data/kaggle_data/test_eda.csv"
train_path = "/content/drive/MyDrive/함께하조/기계학습과 딥러닝/data/kaggle_data/train_eda.csv"

test = pd.read_csv(test_path)
train = pd.read_csv(train_path)

test.drop("Unnamed: 0", axis=1, inplace=True)
train.drop("Unnamed: 0", axis=1, inplace=True)

#cat features
cat_col = ["season", "Year","weather", "Day of week","Month","Day_info"] #Hour
for col in cat_col:
    train[col] = train[col].astype("category")
    test[col] = test[col].astype("category")



#target, drop, y
target_col = "count"
drop_cols = ["datetime", "workingday", "holiday", "Day", "Year", "Hour",target_col] #"sin_hour", "cos_hour"

x_train, y_train = train.drop(drop_cols, axis=1), train[target_col]
x_test = test.drop(["datetime", "workingday", "holiday", "Day", "Year", "Hour"], axis=1)

x_train.head()

,season,weather,temp,humidity,windspeed,Day of week,Month,Day_info,sin_hour,cos_hour
0,1,Good,9.84,81,0.0,Saturday,1,Weekend,0.000000,1.000000
1,1,Good,9.02,80,0.0,Saturday,1,Weekend,0.258819,0.965926
2,1,Good,9.02,80,0.0,Saturday,1,Weekend,0.500000,0.866025
3,1,Good,9.84,75,0.0,Saturday,1,Weekend,0.707107,0.707107
4,1,Good,9.84,75,0.0,Saturday,1,Weekend,0.866025,0.500000


In [ ]:
mcle =  MultiColLabelEncoder()
cat_cols = ["season", "weather", "Day of week", "Month", "Day_info"] 
xtrain_t = mcle.fit_transform(x_train, columns=cat_cols)
xtest_t = mcle.fit_transform(x_test, columns=cat_cols)
xtrain_t.head()

,season,weather,temp,humidity,windspeed,Day of week,Month,Day_info,sin_hour,cos_hour
0,0,0,9.84,81,0.0,2,0,0,0.000000,1.000000
1,0,0,9.02,80,0.0,2,0,0,0.258819,0.965926
2,0,0,9.02,80,0.0,2,0,0,0.500000,0.866025
3,0,0,9.84,75,0.0,2,0,0,0.707107,0.707107
4,0,0,9.84,75,0.0,2,0,0,0.866025,0.500000


In [ ]:
regressor = TabNetRegressor(**tabnet_params)
regressor.fit(xtrain_t.values, np.array(y_train).reshape(-1, 1))

/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [ ]:
y_pred = regressor.predict(xtest_t.values)
y_pred = np.where(y_pred<0, 0, y_pred)

test["count"] = y_pred
submission = test[["datetime", "count"]]
submission.head()

# 0.442
submission.to_csv("TABNetr.csv", index=False)